### Modelo Convolusión 2D - Keras

#### Importación de librerias para la generación del modelo

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

#### Se cargan los datos del csv de las imágenes

In [4]:
train_df = pd.read_csv('boneage-training-dataset.csv')
test_df = pd.read_csv('boneage-test-dataset.csv')

train_df['id'] = train_df['id'].apply(lambda x: str(x)+'.png')
train_df['gender'] = train_df['male'].apply(lambda x: 'male' if x else 'female')

#### Se separa el training data en Training y Validation

In [6]:
df_train, df_valid = train_test_split(train_df, test_size = 0.2, random_state = 0)

#### Se preparan los sets para las imagenes para Train, Validation y Testing

In [7]:
img_size = 256

train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
val_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_data_generator.flow_from_dataframe(
    dataframe = df_train, directory = 'boneage-training-dataset/boneage-training-dataset',
    x_col= 'id', y_col= 'boneage', batch_size = 32, seed = 42,
    shuffle = True, class_mode= 'raw', flip_vertical = True,
    color_mode = 'rgb', target_size = (img_size, img_size))

validation_generator = val_data_generator.flow_from_dataframe(
    dataframe = df_valid, directory = 'boneage-training-dataset/boneage-training-dataset',
    x_col= 'id', y_col= 'boneage', batch_size = 32, seed = 42,
    shuffle = True, class_mode= 'raw', flip_vertical = True,
    color_mode = 'rgb', target_size = (img_size, img_size))

test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_data_generator.flow_from_directory(
    directory = 'boneage-test-dataset',
    shuffle = False, 
    class_mode = None,
    color_mode = 'rgb',
    target_size = (img_size,img_size))

Found 10088 validated image filenames.
Found 2523 validated image filenames.
Found 200 images belonging to 1 classes.


#### Se valida la existencia de las imagenes y se transforman a modo de prueba

In [8]:
test_X, test_Y = next(val_data_generator.flow_from_dataframe( 
                            dataframe = df_valid, directory = 'boneage-training-dataset/boneage-training-dataset',
                            x_col = 'id', y_col = 'boneage', 
                            target_size = (img_size, img_size),
                            batch_size = 2523,
                            class_mode = 'raw'
                            ))

Found 2523 validated image filenames.


#### Generación del modelo y entrenamiento

In [9]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout


model = Sequential()
# Input Layer
model.add(Conv2D(16, kernel_size=(7,7), strides=(2,2), activation='relu', input_shape=(img_size, img_size, 3)))
# Convulotional Layers
model.add(Conv2D(32, kernel_size=(5,5), strides=(2,2), activation='relu'))
# Flattening
model.add(Flatten())
# Dense Layer
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=15)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
315/315 [==============================] - 513s 2s/step - loss: 1902.3429 - mse: 1902.3429 - val_loss: 1554.5613 - val_mse: 1554.5613
Epoch 2/15
315/315 [==============================] - 458s 1s/step - loss: 1398.3236 - mse: 1398.3236 - val_loss: 1352.3217 - val_mse: 1352.3217
Epoch 3/15
315/315 [==============================] - 457s 1s/step - loss: 1308.8787 - mse: 1308.8787 - val_loss: 1238.8737 - val_mse: 1238.8737
Epoch 4/15
315/315 [==============================] - 459s 1s/step - loss: 1119.1768 - mse: 1119.1768 - val_loss: 1193.6454 - val_mse: 1193.6454
Epoch 5/15
315/315 [==============================] - 458s 1s/step - loss: 985.6346 - mse: 985.6346 - val_loss: 1133.5321 - val_mse: 1133.5321
Epoch 6/15
315/315 [==============================] - 457s 1s/step - loss: 923.0568 - mse: 923.0568 - val_loss: 1529.7722 - val_mse: 1529.7722
Epoch 7/15
315/315 [==============================] - 455s

#### Generación de las predicciones con el set de validación

In [33]:
predictions = model.predict(validation_generator)

#### Se realiza una validación para revisar que tan bien funcionó el modelo

In [47]:
print("Edad Osea de la predicción:",predictions[0][0],"años")
print("Edad Osea real:",df_valid['boneage'].iloc[0],"años")


Edad Osea de la predicción: 179.60643 años
Edad Osea real: 94 años
